## Création du filtre de liquidité

- idée : filtre dynamique dans le temps
- À chaque date de rebalancement de référence (dernier jour de trading le janvier, avril, juillet et octobre) on calcule la liquidité moyenne des 3 derniers mois à partir des poids de référence de l'indice et des données de volumes.

In [7]:
import matplotlib.pyplot as plt
from utils import *
from preprocessing import read_data, read_gics_sectors, read_reference_index_holdings

In [8]:
df_px = read_data("Constituents PX_LAST data")
df_volume = read_data("Constituents PX_VOLUME data")
df_total_ret = read_data("Constituents TOT_RET_INDEX data")
df_gics = read_gics_sectors()
df_ref = read_reference_index_holdings()

In [9]:
df_volume.head()

,AAC/B CN Equity,AAH CN Equity,AAR-U CN Equity,AAV CN Equity,ABX CN Equity,AC CN Equity,ACB CN Equity,ACE/A CN Equity,ACE/B CN Equity,ACM/A CN Equity,...,WTE CN Equity,WTN CN Equity,WTO CN Equity,X CN Equity,XG CN Equity,YLO CN Equity,YRI CN Equity,ZL CN Equity,ZZZ CN Equity,SPTSX Index
Index Date,,,,,,,,,,,,,,,,,,,,,
2000-01-04,8750.0,9711.0,NaN,16575.0,926579.0,NaN,NaN,NaN,NaN,21396.0,...,25607.0,NaN,NaN,NaN,NaN,NaN,1804.0,510922.0,NaN,62662940.0
2000-01-05,23285.0,11400.0,NaN,31275.0,1452213.0,NaN,NaN,NaN,NaN,6000.0,...,55081.0,NaN,NaN,NaN,NaN,NaN,3357.0,343030.0,NaN,94698130.0
2000-01-06,29305.0,8783.0,NaN,5150.0,600273.0,NaN,NaN,NaN,NaN,38400.0,...,23580.0,NaN,NaN,NaN,NaN,NaN,2107.0,518189.0,NaN,84015360.0
2000-01-07,13695.0,11620.0,NaN,4555.0,1020813.0,NaN,NaN,NaN,NaN,3400.0,...,34934.0,NaN,NaN,NaN,NaN,NaN,891.0,202823.0,NaN,84001320.0
2000-01-10,16070.0,20453.0,NaN,8384.0,1269818.0,NaN,NaN,NaN,NaN,71800.0,...,19667.0,NaN,NaN,NaN,NaN,NaN,2249.0,278023.0,NaN,78874940.0


In [14]:
df_px.head()

,AAC/B CN Equity,AAH CN Equity,AAR-U CN Equity,AAV CN Equity,ABX CN Equity,AC CN Equity,ACB CN Equity,ACE/A CN Equity,ACE/B CN Equity,ACM/A CN Equity,...,WTE CN Equity,WTN CN Equity,WTO CN Equity,X CN Equity,XG CN Equity,YLO CN Equity,YRI CN Equity,ZL CN Equity,ZZZ CN Equity,SPTSX Index
Index Date,,,,,,,,,,,,,,,,,,,,,
2000-01-04,12.00,12.75,NaN,6.0,25.45,NaN,NaN,NaN,NaN,13.50,...,4.9332,NaN,NaN,NaN,NaN,NaN,11.5505,19.45,NaN,8202.60
2000-01-05,12.25,12.55,NaN,6.0,25.65,NaN,NaN,NaN,NaN,14.00,...,4.9830,NaN,NaN,NaN,NaN,NaN,10.8630,18.60,NaN,8119.42
2000-01-06,12.50,12.55,NaN,6.0,25.80,NaN,NaN,NaN,NaN,13.50,...,4.9830,NaN,NaN,NaN,NaN,NaN,10.7255,18.90,NaN,8114.24
2000-01-07,12.55,12.25,NaN,6.0,25.25,NaN,NaN,NaN,NaN,13.75,...,4.9332,NaN,NaN,NaN,NaN,NaN,11.5505,19.00,NaN,8429.42
2000-01-10,13.35,13.00,NaN,5.8,25.25,NaN,NaN,NaN,NaN,13.50,...,4.9332,NaN,NaN,NaN,NaN,NaN,11.5505,19.00,NaN,8602.50


In [15]:
df_total_ret.head()

,AAC/B CN Equity,AAH CN Equity,AAR-U CN Equity,AAV CN Equity,ABX CN Equity,AC CN Equity,ACB CN Equity,ACE/A CN Equity,ACE/B CN Equity,ACM/A CN Equity,...,WTE CN Equity,WTN CN Equity,WTO CN Equity,X CN Equity,XG CN Equity,YLO CN Equity,YRI CN Equity,ZL CN Equity,ZZZ CN Equity,SPTSX Index
Index Date,,,,,,,,,,,,,,,,,,,,,
2000-01-04,12.00,12.75,NaN,6.0,25.45,NaN,NaN,NaN,NaN,13.50,...,4.9332,NaN,NaN,NaN,NaN,NaN,11.5505,19.45,NaN,8202.6000
2000-01-05,12.25,12.55,NaN,6.0,25.65,NaN,NaN,NaN,NaN,14.00,...,4.9830,NaN,NaN,NaN,NaN,NaN,10.8630,18.60,NaN,8119.5094
2000-01-06,12.50,12.55,NaN,6.0,25.80,NaN,NaN,NaN,NaN,13.50,...,4.9830,NaN,NaN,NaN,NaN,NaN,10.7255,18.90,NaN,8114.3695
2000-01-07,12.55,12.25,NaN,6.0,25.25,NaN,NaN,NaN,NaN,13.75,...,4.9332,NaN,NaN,NaN,NaN,NaN,11.5505,19.00,NaN,8429.5545
2000-01-10,13.35,13.00,NaN,5.8,25.25,NaN,NaN,NaN,NaN,13.50,...,4.9332,NaN,NaN,NaN,NaN,NaN,11.5505,19.00,NaN,8602.6765


In [16]:
df_gics.head()

,Ticker,GICS Sector
0,AAC/B CN Equity,None
1,AAH CN Equity,None
2,AAR-U CN Equity,Real Estate
3,AAV CN Equity,Energy
4,ABX CN Equity,Materials


In [17]:
df_ref.head()

,Ticker,Weight
Index Date,,
2002-01-31,0112977D CT,0.056640
2002-01-31,0132533D CT,2.797760
2002-01-31,0206904D CT,0.705212
2002-01-31,0227217Q CT,0.140393
2002-01-31,0334249Q CT,0.501815


In [10]:
df_volume.shape, df_px.shape, df_total_ret.shape, df_gics.shape, df_ref.shape

((5810, 573), (5810, 573), (5830, 573), (572, 2), (61152, 2))

In [13]:
pd.read_parquet("converted_data/filtered_data.parquet")

,AAC/B CN Equity,AAH CN Equity,AAR-U CN Equity,AAV CN Equity,ABX CN Equity,AC CN Equity,ACB CN Equity,ACE/A CN Equity,ACE/B CN Equity,ACM/A CN Equity,...,WSP CN Equity,WTE CN Equity,WTN CN Equity,WTO CN Equity,X CN Equity,XG CN Equity,YLO CN Equity,YRI CN Equity,ZL CN Equity,ZZZ CN Equity
Index Date,,,,,,,,,,,,,,,,,,,,,
2000-01-04,8750.0,9711.0,NaN,16575.0,926579.0,NaN,NaN,NaN,NaN,21396.0,...,NaN,25607.0,NaN,NaN,NaN,NaN,NaN,NaN,510922.0,NaN
2000-01-05,23285.0,11400.0,NaN,31275.0,1452213.0,NaN,NaN,NaN,NaN,6000.0,...,NaN,55081.0,NaN,NaN,NaN,NaN,NaN,NaN,343030.0,NaN
2000-01-06,29305.0,8783.0,NaN,5150.0,600273.0,NaN,NaN,NaN,NaN,38400.0,...,NaN,23580.0,NaN,NaN,NaN,NaN,NaN,NaN,518189.0,NaN
2000-01-07,13695.0,11620.0,NaN,4555.0,1020813.0,NaN,NaN,NaN,NaN,3400.0,...,NaN,34934.0,NaN,NaN,NaN,NaN,NaN,NaN,202823.0,NaN
2000-01-10,16070.0,20453.0,NaN,8384.0,1269818.0,NaN,NaN,NaN,NaN,71800.0,...,NaN,19667.0,NaN,NaN,NaN,NaN,NaN,NaN,278023.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-22,NaN,NaN,NaN,1228248.0,6959963.0,2948587.0,3467182.0,NaN,NaN,NaN,...,199407.0,162937.0,NaN,NaN,55049.0,NaN,NaN,1676407.0,NaN,160819.0
2022-12-23,NaN,NaN,NaN,898735.0,6510750.0,2199482.0,1811017.0,NaN,NaN,NaN,...,123579.0,114166.0,NaN,NaN,79987.0,NaN,NaN,2852294.0,NaN,161553.0
2022-12-28,NaN,NaN,NaN,1477605.0,10668831.0,4472163.0,4727413.0,NaN,NaN,NaN,...,184946.0,142018.0,NaN,NaN,92335.0,NaN,NaN,1810390.0,NaN,68716.0
